<div style="background-color: black; padding: 20px; text-align:center">
    <img src="image/openclassrooms.png" alt="OpenClassrooms" style="display:block; margin: 0 auto; width: 300px;"><br>  
    <span style="font-size: 36px; font-family: 'Lucida Console', Monaco, monospace; font-weight: bold; color: white;display: block;">
        PROJET n°12 : Détectez des faux billets avec R ou <img src="image/python.png" alt="Python" style="width: auto; height: 36px; vertical-align:middle;"> <!-- Hauteur fixe pour Python, aligné au texte -->
    </span>
</div>

<div style="background-color: black; padding: 20px;">
<span style="display: flex; margin: auto; font-size: 36px; font-family: 'Lucida Console', Monaco, monospace; font-weight: bold; color: white;">

</span>

* [<span style="margin: auto; color: #39FF14; font-size: 28px; font-family: 'Lucida Console', Monaco, monospace; font-weight: bold;">5 - APPLICATION FONCTIONNELLE</span>](#chapter5)
</div>

In [1]:
# Importation des bibliothèques standards

import numpy as np
import pandas as pd
import os
import joblib

# Assurez-vous que les bibliothèques suivantes sont installées :
# - numpy
# - pandas
# - joblib

# Configuration du chemin pour les modèles
if not os.path.exists('models'):
    raise FileNotFoundError("Le répertoire 'models' contenant les modèles sauvegardés n'existe pas.")

In [2]:
# === PRÉDICTIONS SUR NOUVELLES DONNÉES ===

def predict_production(data_path, model_name='random_forest'):
    """
    Fait des prédictions sur de nouvelles données.
    """
    try:
        # Lire les données avec le séparateur correct
        new_data = pd.read_csv(data_path, sep=',')  # séparateur virgule pour les données de production
        print(f"Données chargées : {len(new_data)} lignes")
        
        # Définir l'ordre exact des features comme dans l'entraînement
        base_features = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']
        
        print("\nVérification des colonnes :")
        print("Colonnes attendues :", base_features)
        print("Colonnes présentes :", new_data.columns.tolist())
        
        # Vérifier la présence de toutes les colonnes nécessaires
        for feature in base_features:
            if feature not in new_data.columns:
                raise ValueError(f"Colonne manquante : {feature}")
        
        # Créer les features dérivées
        print("\nCréation des features dérivées...")
        new_data['height_ratio'] = new_data['height_left'] / new_data['height_right']
        new_data['margin_ratio'] = new_data['margin_up'] / new_data['margin_low']
        new_data['diagonal_length_ratio'] = new_data['diagonal'] / new_data['length']
        
        # Sélectionner et ordonner les features dans le même ordre que l'entraînement
        features = base_features + ['height_ratio', 'margin_ratio', 'diagonal_length_ratio']
        X = new_data[features]
        
        print("\nDimensions des données avant scaling :", X.shape)
        
        # Charger le scaler et normaliser les données
        scaler = joblib.load(os.path.join('models', 'scaler.pkl'))
        X_scaled = scaler.transform(X)
        print("Dimensions des données après scaling :", X_scaled.shape)
        
        # Charger le modèle et faire les prédictions
        model = joblib.load(os.path.join('models', model_name, f"{model_name}.pkl"))
        predictions = model.predict(X_scaled)
        probabilities = model.predict_proba(X_scaled)
        
        # Créer le DataFrame des résultats
        results = pd.DataFrame({
            'ID': new_data['id'],
            'Prédiction': predictions,
            'Classe': ['Authentique' if p == 1 else 'Contrefaçon' for p in predictions],
            'Probabilité_Authentique': [f"{prob:.1f}%" for prob in (probabilities[:, 1] * 100)]
        })
        
        # Afficher résumé
        print("\n=== RÉSUMÉ DES PRÉDICTIONS ===")
        print(f"Nombre total de billets : {len(results)}")
        
        authentiques = (results['Classe'] == 'Authentique').sum()
        contrefaits = (results['Classe'] == 'Contrefaçon').sum()
        
        print(f"\nBillets authentiques : {authentiques} ({authentiques/len(results)*100:.1f}%)")
        auth_results = results[results['Classe'] == 'Authentique']
        print("\nDétail des billets authentiques :")
        print(auth_results[['ID', 'Probabilité_Authentique']].to_string(index=False))
        
        print(f"\nBillets contrefaits : {contrefaits} ({contrefaits/len(results)*100:.1f}%)")
        print("\nDétail des billets contrefaits :")
        fake_results = results[results['Classe'] == 'Contrefaçon']
        print(fake_results[['ID', 'Probabilité_Authentique']].to_string(index=False))
        
        print("\n=== RÉSULTATS COMPLETS ===")
        print(results.to_string(index=False))
        
        return results
        
    except Exception as e:
        print(f"Une erreur s'est produite : {str(e)}")
        raise

In [5]:
# Utilisation
if __name__ == "__main__":
    
    data_path = 'billets_production.csv'  # Remplacez par le chemin de votre fichier de données

    predictions = predict_production(data_path)

Données chargées : 5 lignes

Vérification des colonnes :
Colonnes attendues : ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']
Colonnes présentes : ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length', 'id']

Création des features dérivées...

Dimensions des données avant scaling : (5, 9)
Dimensions des données après scaling : (5, 9)

=== RÉSUMÉ DES PRÉDICTIONS ===
Nombre total de billets : 5

Billets authentiques : 2 (40.0%)

Détail des billets authentiques :
 ID Probabilité_Authentique
A_4                  100.0%
A_5                  100.0%

Billets contrefaits : 3 (60.0%)

Détail des billets contrefaits :
 ID Probabilité_Authentique
A_1                   -0.0%
A_2                   -0.0%
A_3                   -0.0%

=== RÉSULTATS COMPLETS ===
 ID  Prédiction      Classe Probabilité_Authentique
A_1           0 Contrefaçon                   -0.0%
A_2           0 Contrefaçon                   -0.0%
A_3           0 Contrefaçon  

c:\mamba\envs\Mamba_GPU_3_9\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\mamba\envs\Mamba_GPU_3_9\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
